In [2]:
import json
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
load_dotenv()
import os
import praw
import pandas as pd


In [3]:
import praw
import pandas as pd

# Configure Reddit API credentials
reddit = praw.Reddit(
    client_id="1NXEdgwQdxth3R1--kczsA",         # Replace with your client_id
    client_secret="TCRwQ5olTeV5TlAblY6oDLEn-PkWfA", # Replace with your client_secret
    user_agent="MyScraper v1.0 by /u/financialdata"        # Replace with your user_agent
)

# Define the subreddit and data collection parameters
subreddit = reddit.subreddit("stocks")
post_limit = 50000  # Number of posts to fetch

# Initialize a list to hold the data
data = []

# Scrape posts
print("Scraping data from r/stocks...")
for post in subreddit.hot(limit=post_limit):  # Use 'hot', 'new', or 'top' as needed
    data.append({
        "Post ID": post.id,
        "Title": post.title,
        "Score": post.score,
        "Upvote Ratio": post.upvote_ratio,
        "Number of Comments": post.num_comments,
        "Created At": post.created_utc,  # UTC timestamp
        "Author": post.author.name if post.author else "Deleted",
        "Post URL": post.url,
        "Content": post.selftext  # The post body (text content)
    })

# Convert data to a pandas DataFrame
df = pd.DataFrame(data)

# Save data to a CSV file
csv_file = "reddit_stocks_data.csv"
df.to_csv(csv_file, index=False)

print(f"Scraping complete! Data saved to {csv_file}.")


Scraping data from r/stocks...
Scraping complete! Data saved to reddit_stocks_data.csv.


In [12]:
data = pd.read_csv("reddit_stocks_data.csv", encoding="ISO-8859-1")
data.head()

,Post ID,Title,Score,Upvote Ratio,Number of Comments,Created At,Author,Post URL,Content
0,1grtfyq,r/Stocks Daily Discussion & Fundamentals Frida...,25,0.94,477,1.731667e+09,AutoModerator,https://www.reddit.com/r/stocks/comments/1grtf...,"This is the daily discussion, so anything stoc..."
1,1h39vdi,/r/Stocks Weekend Discussion Saturday - Nov 30...,2,1.00,9,1.732963e+09,AutoModerator,https://www.reddit.com/r/stocks/comments/1h39v...,This is the weekend edition of our stickied di...
2,1h3465k,Do you think Rivian would make it to $100 a sh...,67,0.60,258,1.732940e+09,CoolTomatoh,https://www.reddit.com/r/stocks/comments/1h346...,"With talks of Rivian saving VW, California kee..."
3,1h2roez,23-Hour Party People: U.S. stocks may soon tra...,407,0.92,162,1.732903e+09,DecisionNo1902,https://www.reddit.com/r/stocks/comments/1h2ro...,**23-Hour Party People:** \n**U.S. stocks may...
4,1h2nht8,Thoughts on buying BRB.K this month instead of...,56,0.80,51,1.732892e+09,Life-Ambition1432,https://www.reddit.com/r/stocks/comments/1h2nh...,Does anyone have any thoughts on buying BRB.K ...


In [13]:
# ! pip install TextBlob
import pandas as pd
from textblob import TextBlob
import re


In [14]:
# Function to calculate sentiment polarity
def analyze_sentiment(text):
    analysis = TextBlob(str(text))
    return analysis.sentiment.polarity  # Range: -1 to +1

# Apply sentiment analysis to titles and content
data['Title Sentiment'] = data['Title'].apply(analyze_sentiment)
data['Content Sentiment'] = data['Content'].apply(analyze_sentiment)


In [15]:
# Function to extract stock mentions (e.g., ticker symbols)
def extract_stock_mentions(text):
    # Assuming stock tickers are all-caps (1-4 letters, e.g., TSLA, GOOG)
    return re.findall(r'\b[A-Z]{1,4}\b', str(text))

# Count the number of stock mentions in each post
data['Stock Mentions'] = data['Content'].apply(lambda x: len(extract_stock_mentions(x)))


In [16]:
# Display the first few rows with the calculated features
print(data[['Title', 'Title Sentiment', 'Content Sentiment', 'Stock Mentions']].head())


                                               Title  Title Sentiment  \
0  r/Stocks Daily Discussion & Fundamentals Frida...              0.0   
1  /r/Stocks Weekend Discussion Saturday - Nov 30...              0.0   
2  Do you think Rivian would make it to $100 a sh...              0.0   
3  23-Hour Party People: U.S. stocks may soon tra...              0.1   
4  Thoughts on buying BRB.K this month instead of...              0.0   

   Content Sentiment  Stock Mentions  
0           0.036523              16  
1          -0.050000               2  
2           0.100000               2  
3           0.007273               5  
4           0.042188              19  


In [17]:
# Save the updated dataframe to a new CSV file
output_path = 'processed_reddit_stocks_data.csv'
data.to_csv(output_path, index=False)
